# 🧬 Algoritma Genetika untuk Penentuan Kelompok KKM Reguler
## UIN Malang - Kaggle Version

---

**Platform**: Kaggle Notebook  
**Dataset**: Upload `master_data.csv` ke Kaggle Dataset

**Tujuan**: Mengelompokkan mahasiswa ke dalam kelompok-kelompok KKM Reguler yang optimal dengan mempertimbangkan:
- ✅ Keberadaan anggota HTQ
- ✅ Heterogenitas jurusan
- ✅ Proporsi jenis kelamin
- ✅ Jumlah anggota per kelompok

**Metode**: Genetic Algorithm dengan PMX Crossover dan Reciprocal Exchange Mutation

---

### 📋 Langkah Setup di Kaggle:
1. Upload dataset `master_data.csv` ke Kaggle Dataset
2. Add dataset ke notebook ini
3. Run all cells
4. Download hasil dari Output section

## 1. Import Libraries & Setup Environment

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime
import os
import glob

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"   Pandas: {pd.__version__}")
print(f"   Numpy: {np.__version__}")

# Detect environment (Kaggle vs Local)
if os.path.exists('/kaggle/input'):
    print("🌐 Running on KAGGLE environment")
    KAGGLE_MODE = True
    INPUT_DIR = '/kaggle/input'
    OUTPUT_DIR = '/kaggle/working'
else:
    print("💻 Running on LOCAL environment")
    KAGGLE_MODE = False
    INPUT_DIR = '../data'
    OUTPUT_DIR = '../output'

print(f"   Input directory: {INPUT_DIR}")
print(f"   Output directory: {OUTPUT_DIR}")

# Create output directory structure for testing scenarios
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f'{OUTPUT_DIR}/skenario_results', exist_ok=True)
os.makedirs(f'{OUTPUT_DIR}/summary', exist_ok=True)
os.makedirs(f'{OUTPUT_DIR}/plots', exist_ok=True)

print(f"   📁 Output structure:")
print(f"      - {OUTPUT_DIR}/skenario_results (per-run results)")
print(f"      - {OUTPUT_DIR}/summary (aggregated statistics)")
print(f"      - {OUTPUT_DIR}/plots (comparison charts)")

✅ Libraries imported successfully!
   Pandas: 2.3.3
   Numpy: 2.3.4
💻 Running on LOCAL environment
   Input directory: ../data
   Output directory: ../output
   📁 Output structure:
      - ../output/skenario_results (per-run results)
      - ../output/summary (aggregated statistics)
      - ../output/plots (comparison charts)


## 2. Load and Validate Data

In [2]:
# Auto-detect CSV file in Kaggle or local
if KAGGLE_MODE:
    # Find CSV file in Kaggle input
    csv_files = glob.glob(f'{INPUT_DIR}/**/*.csv', recursive=True)
    if csv_files:
        csv_path = csv_files[0]
        print(f"📁 Found dataset: {csv_path}")
    else:
        raise FileNotFoundError("No CSV file found in Kaggle input. Please add dataset!")
else:
    csv_path = f'{INPUT_DIR}/master_data.csv'

# Load data
df = pd.read_csv(csv_path)

# Validate required columns
required_cols = ['ID', 'Jenis Kelamin', 'Jurusan', 'HTQ']
assert all(col in df.columns for col in required_cols), f"Missing columns! Required: {required_cols}"

# Check missing values
missing_count = df[required_cols].isnull().sum().sum()
assert missing_count == 0, f"Found {missing_count} missing values!"

# Check duplicate IDs
dup_count = df['ID'].duplicated().sum()
assert dup_count == 0, f"Found {dup_count} duplicate IDs!"

print("="*80)
print("✅ DATA VALIDATION PASSED")
print("="*80)
print(f"Total Mahasiswa: {len(df)}")
print(f"Jumlah Jurusan: {df['Jurusan'].nunique()}")
print(f"\nDistribusi Jenis Kelamin:")
print(df['Jenis Kelamin'].value_counts())
print(f"\nDistribusi HTQ:")
print(df['HTQ'].value_counts())
print(f"\nTop 5 Jurusan:")
print(df['Jurusan'].value_counts().head())
print("\nSample Data:")
df.head(10)

✅ DATA VALIDATION PASSED
Total Mahasiswa: 2338
Jumlah Jurusan: 24

Distribusi Jenis Kelamin:
Jenis Kelamin
PR    1391
LK     947
Name: count, dtype: int64

Distribusi HTQ:
HTQ
Tidak    2112
Ya        226
Name: count, dtype: int64

Top 5 Jurusan:
Jurusan
MANAJEMEN                    248
PSIKOLOGI                    218
BAHASA DAN SASTRA ARAB       184
BAHASA DAN SASTRA INGGRIS    183
HUKUM BISNIS SYARI'AH        177
Name: count, dtype: int64

Sample Data:


,ID,Jenis Kelamin,Jurusan,HTQ
0,1,PR,BAHASA DAN SASTRA INGGRIS,Tidak
1,2,PR,BIOLOGI,Tidak
2,3,PR,TEKNIK INFORMATIKA,Tidak
3,4,LK,BAHASA DAN SASTRA ARAB,Tidak
4,5,LK,ILMU AL-QUR`AN DAN TAFSIR,Tidak
5,6,PR,AL-AHWAL AL-SYAKHSHIYYAH,Tidak
6,7,PR,PSIKOLOGI,Tidak
7,8,PR,PERBANKAN SYARI`AH,Tidak
8,9,LK,HUKUM BISNIS SYARI'AH,Tidak
9,10,LK,MANAJEMEN,Tidak


## 3. Data Preprocessing

In [3]:
def preprocess_data(df, jumlah_kelompok):
    """Preprocess data dan hitung semua statistik yang diperlukan"""
    df_clean = df.copy()
    
    # Normalize HTQ to binary
    df_clean['HTQ'] = df_clean['HTQ'].apply(lambda x: 1 if str(x).lower() in ['ya', 'lulus', '1'] else 0)
    
    # Calculate aggregate statistics
    N = len(df_clean)
    L = (df_clean['Jenis Kelamin'] == 'LK').sum()
    P = (df_clean['Jenis Kelamin'] == 'PR').sum()
    K = jumlah_kelompok
    
    # Calculate expected proportions
    PL = L / N
    PP = P / N
    
    # Calculate expected sizes per group
    A = N // K
    sisa = N % K
    
    expected_sizes = [A + 1 if i < sisa else A for i in range(K)]
    
    # Max fitness
    max_fitness = K * 4
    
    return {
        'df_clean': df_clean, 'N': N, 'L': L, 'P': P, 'K': K,
        'PL': PL, 'PP': PP, 'A': A, 'sisa': sisa,
        'expected_sizes': expected_sizes, 'max_fitness': max_fitness
    }

# Set jumlah kelompok - ADJUST THIS VALUE
JUMLAH_KELOMPOK = 190

# Preprocess
preprocessed = preprocess_data(df, JUMLAH_KELOMPOK)
df_clean = preprocessed['df_clean']

print("="*80)
print("✅ PREPROCESSING COMPLETE")
print("="*80)
print(f"Total Mahasiswa (N): {preprocessed['N']}")
print(f"Laki-laki (L): {preprocessed['L']} ({preprocessed['PL']:.2%})")
print(f"Perempuan (P): {preprocessed['P']} ({preprocessed['PP']:.2%})")
print(f"Jumlah Kelompok (K): {preprocessed['K']}")
print(f"Base size: {preprocessed['A']}, Sisa: {preprocessed['sisa']}")
print(f"Expected sizes: {preprocessed['expected_sizes'][:5]}... (first 5)")
print(f"Max Fitness: {preprocessed['max_fitness']}")

✅ PREPROCESSING COMPLETE
Total Mahasiswa (N): 2338
Laki-laki (L): 947 (40.50%)
Perempuan (P): 1391 (59.50%)
Jumlah Kelompok (K): 190
Base size: 12, Sisa: 58
Expected sizes: [13, 13, 13, 13, 13]... (first 5)
Max Fitness: 760


## 4. Constraint Evaluation Functions

In [4]:
def evaluate_C1(group_df):
    """C1: Minimal ada 1 anggota HTQ di kelompok"""
    htq_count = group_df['HTQ'].sum()
    return 1 if htq_count >= 1 else 0

def evaluate_C2(group_df):
    """C2: Jumlah jurusan berbeda > 50% dari ukuran kelompok"""
    unique_majors = group_df['Jurusan'].nunique()
    threshold = len(group_df) * 0.5
    return 1 if unique_majors > threshold else 0

def evaluate_C3(group_df, PL, PP):
    """C3: Proporsi gender menyimpang ±10% dari proporsi ideal"""
    n_group = len(group_df)
    lk_count = (group_df['Jenis Kelamin'] == 'LK').sum()
    pr_count = (group_df['Jenis Kelamin'] == 'PR').sum()
    
    lk_prop = lk_count / n_group
    pr_prop = pr_count / n_group
    
    lk_dev = abs(lk_prop - PL)
    pr_dev = abs(pr_prop - PP)
    
    return 1 if (lk_dev <= 0.1 and pr_dev <= 0.1) else 0

def evaluate_C4(group_df, expected_size):
    """C4: Ukuran kelompok sesuai expected size"""
    return 1 if len(group_df) == expected_size else 0

print("✅ Constraint functions defined (C1, C2, C3, C4)")

✅ Constraint functions defined (C1, C2, C3, C4)


## 5. Fitness Calculation

In [5]:
def decode_kromosom(kromosom, df_clean, expected_sizes):
    """Decode permutation kromosom into groups"""
    groups = []
    start_idx = 0
    
    for i, size in enumerate(expected_sizes):
        end_idx = start_idx + size
        group_ids = kromosom[start_idx:end_idx]
        group_df = df_clean[df_clean['ID'].isin(group_ids)].copy()
        groups.append(group_df)
        start_idx = end_idx
    
    return groups

def calculate_fitness(kromosom, df_clean, expected_sizes, PL, PP):
    """Calculate total fitness of a kromosom"""
    groups = decode_kromosom(kromosom, df_clean, expected_sizes)
    total_fitness = 0
    
    for i, group_df in enumerate(groups):
        c1 = evaluate_C1(group_df)
        c2 = evaluate_C2(group_df)
        c3 = evaluate_C3(group_df, PL, PP)
        c4 = evaluate_C4(group_df, expected_sizes[i])
        
        total_fitness += (c1 + c2 + c3 + c4)
    
    return total_fitness

print("✅ Fitness calculation functions defined")

✅ Fitness calculation functions defined


## 6. Population Initialization

In [6]:
def initialize_population(df_clean, popsize):
    """Initialize population with random permutations"""
    student_ids = df_clean['ID'].values
    population = []
    
    for _ in range(popsize):
        kromosom = np.random.permutation(student_ids)
        population.append(kromosom)
    
    return population

print("✅ Population initialization function defined")

✅ Population initialization function defined


## 7. Parent Selection

In [7]:
def select_parents_for_crossover(population, cr):
    """Select parent pairs for crossover based on CR"""
    num_crossover = int(len(population) * cr)
    if num_crossover % 2 != 0:
        num_crossover += 1
    
    # Need at least 2 individuals for crossover
    if num_crossover < 2 or len(population) < 2:
        return []
    
    # Can't select more than population size
    num_crossover = min(num_crossover, len(population))
    
    indices = np.random.choice(len(population), num_crossover, replace=False)
    parent_pairs = [(population[indices[i]], population[indices[i+1]]) 
                    for i in range(0, num_crossover, 2)]
    return parent_pairs

def select_parents_for_mutation(population, mr):
    """Select parents for mutation based on MR"""
    num_mutation = int(len(population) * mr)
    
    # Handle edge cases
    if num_mutation == 0 or len(population) == 0:
        return []
    
    num_mutation = min(num_mutation, len(population))
    indices = np.random.choice(len(population), num_mutation, replace=False)
    return [population[i] for i in indices]

print("✅ Parent selection functions defined")

✅ Parent selection functions defined


## 8. PMX Crossover

In [8]:
def pmx_crossover(parent1, parent2):
    """
    Partially Mapped Crossover (PMX) - Fixed version
    Prevents infinite loops by following the mapping chain properly
    """
    size = len(parent1)
    
    # Choose two random cut points
    cx_point1 = np.random.randint(0, size)
    cx_point2 = np.random.randint(0, size)
    if cx_point1 > cx_point2:
        cx_point1, cx_point2 = cx_point2, cx_point1
    
    # Ensure we have at least some segment to swap
    if cx_point1 == cx_point2:
        cx_point2 = min(cx_point1 + 1, size)
    
    # Initialize offspring as copies
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    # Swap middle segments
    child1[cx_point1:cx_point2] = parent2[cx_point1:cx_point2]
    child2[cx_point1:cx_point2] = parent1[cx_point1:cx_point2]
    
    # Fix conflicts using proper PMX algorithm
    def fix_conflicts_pmx(child, p1, p2, start, end):
        """
        Fix conflicts by following the mapping relationship.
        For each position outside the crossover segment,
        if there's a conflict, follow the mapping chain until finding a valid value.
        """
        # Create a set of values in the middle segment for fast lookup
        middle_values = set(child[start:end])
        
        for i in range(size):
            # Only fix positions outside the crossover segment
            if i < start or i >= end:
                # If current value is already in the middle segment (conflict)
                if child[i] in middle_values:
                    # Follow the mapping chain to find a valid replacement
                    value = child[i]
                    visited = set()  # Prevent infinite loops in case of cycles
                    
                    # Keep following the mapping until we find a value not in middle segment
                    while value in middle_values and value not in visited:
                        visited.add(value)
                        
                        # Find where this value appears in p2's middle segment
                        try:
                            idx_in_p2 = np.where(p2[start:end] == value)[0][0] + start
                            # Get the corresponding value from p1
                            value = p1[idx_in_p2]
                        except (IndexError, TypeError):
                            # If not found, break to avoid error
                            break
                    
                    # If we found a valid value (not in middle), use it
                    if value not in middle_values:
                        child[i] = value
                    # else: keep original value (shouldn't happen in valid permutation)
    
    fix_conflicts_pmx(child1, parent1, parent2, cx_point1, cx_point2)
    fix_conflicts_pmx(child2, parent2, parent1, cx_point1, cx_point2)
    
    return child1, child2

print("✅ PMX Crossover function defined (fixed infinite loop bug)")

✅ PMX Crossover function defined (fixed infinite loop bug)


## 9. Reciprocal Exchange Mutation

In [9]:
def reciprocal_exchange_mutation(parent):
    """Swap two random genes"""
    child = parent.copy()
    idx1, idx2 = np.random.choice(len(child), 2, replace=False)
    child[idx1], child[idx2] = child[idx2], child[idx1]
    return child

print("✅ Reciprocal Exchange Mutation function defined")

✅ Reciprocal Exchange Mutation function defined


## 10. Elitism Replacement Strategy

In [10]:
def elitism_replacement(population, offspring, df_clean, expected_sizes, PL, PP, popsize):
    """Replace population with best individuals from combined pool"""
    combined = population + offspring
    
    # Calculate fitness for all
    fitness_scores = [calculate_fitness(ind, df_clean, expected_sizes, PL, PP) 
                      for ind in combined]
    
    # Sort by fitness (descending)
    sorted_indices = np.argsort(fitness_scores)[::-1]
    
    # Select top PopSize individuals
    new_population = [combined[i] for i in sorted_indices[:popsize]]
    new_fitness = [fitness_scores[i] for i in sorted_indices[:popsize]]
    
    return new_population, new_fitness

def elitism_replacement_optimized(population, population_fitness, offspring, 
                                   df_clean, expected_sizes, PL, PP, popsize):
    """
    Optimized elitism with fitness caching.
    Only calculates fitness for NEW offspring, reuses existing population fitness.
    This dramatically speeds up the algorithm (6× faster per generation).
    """
    # Calculate fitness ONLY for new offspring
    offspring_fitness = [calculate_fitness(ind, df_clean, expected_sizes, PL, PP) 
                        for ind in offspring]
    
    # Combine populations and fitness scores
    combined = population + offspring
    combined_fitness = population_fitness + offspring_fitness
    
    # Sort by fitness (descending)
    sorted_indices = sorted(range(len(combined)), 
                          key=lambda i: combined_fitness[i], 
                          reverse=True)
    
    # Select top PopSize individuals
    new_population = [combined[i] for i in sorted_indices[:popsize]]
    new_fitness = [combined_fitness[i] for i in sorted_indices[:popsize]]
    
    return new_population, new_fitness

print("✅ Elitism Replacement functions defined (standard & optimized)")

✅ Elitism Replacement functions defined (standard & optimized)


## 11. Define GA Runner Function

In [11]:
import time

def run_ga_single(df_clean, preprocessed, popsize, max_gen, cr, mr, seed, target_fitness=1.0):
    """
    Run single GA experiment with given parameters
    
    Returns:
        dict: Results containing fitness, generation, runtime, etc.
    """
    # Set seed for reproducibility
    np.random.seed(seed)
    random.seed(seed)
    
    # Extract preprocessed data
    N = preprocessed['N']
    K = preprocessed['K']
    PL = preprocessed['PL']
    PP = preprocessed['PP']
    expected_sizes = preprocessed['expected_sizes']
    max_fitness = preprocessed['max_fitness']
    
    # Initialize
    start_time = time.time()
    population = initialize_population(df_clean, popsize)
    
    # Calculate initial fitness
    init_fitness_start = time.time()
    population_fitness = []
    for kromosom in population:
        fitness = calculate_fitness(kromosom, df_clean, expected_sizes, PL, PP)
        population_fitness.append(fitness)
    init_fitness_time = time.time() - init_fitness_start
    
    # Track best solution
    best_fitness_history = []
    avg_fitness_history = []
    best_overall_fitness = max(population_fitness)
    best_overall_solution = population[population_fitness.index(best_overall_fitness)].copy()
    
    # Main GA Loop
    generation = 0
    for generation in range(1, max_gen + 1):
        # Crossover
        parent_pairs = select_parents_for_crossover(population, cr)
        offspring_cx = []
        for p1, p2 in parent_pairs:
            c1, c2 = pmx_crossover(p1, p2)
            offspring_cx.extend([c1, c2])
        
        # Mutation
        parents_mut = select_parents_for_mutation(population, mr)
        offspring_mut = [reciprocal_exchange_mutation(p) for p in parents_mut]
        
        # Combine offspring
        offspring = offspring_cx + offspring_mut
        
        # Replacement
        population, population_fitness = elitism_replacement_optimized(
            population, population_fitness, offspring, 
            df_clean, expected_sizes, PL, PP, popsize
        )
        
        # Track statistics
        best_fitness = population_fitness[0]
        avg_fitness = np.mean(population_fitness)
        best_fitness_history.append(best_fitness)
        avg_fitness_history.append(avg_fitness)
        
        # Update best overall
        if best_fitness > best_overall_fitness:
            best_overall_fitness = best_fitness
            best_overall_solution = population[0].copy()
        
        # Check termination
        if best_fitness >= target_fitness * max_fitness:
            break
    
    # Final results
    total_time = time.time() - start_time
    
    # Calculate constraint satisfaction
    best_groups = decode_kromosom(best_overall_solution, df_clean, expected_sizes)
    constraint_stats = {
        'C1_satisfied': 0, 'C2_satisfied': 0, 
        'C3_satisfied': 0, 'C4_satisfied': 0, 
        'perfect_groups': 0
    }
    
    for i, group_df in enumerate(best_groups):
        c1 = evaluate_C1(group_df)
        c2 = evaluate_C2(group_df)
        c3 = evaluate_C3(group_df, PL, PP)
        c4 = evaluate_C4(group_df, expected_sizes[i])
        
        constraint_stats['C1_satisfied'] += c1
        constraint_stats['C2_satisfied'] += c2
        constraint_stats['C3_satisfied'] += c3
        constraint_stats['C4_satisfied'] += c4
        
        if c1 + c2 + c3 + c4 == 4:
            constraint_stats['perfect_groups'] += 1
    
    return {
        'seed': seed,
        'best_fitness': best_overall_fitness,
        'best_fitness_pct': best_overall_fitness / max_fitness,
        'final_generation': generation,
        'initial_best_fitness': best_fitness_history[0] if best_fitness_history else 0,
        'fitness_improvement': best_overall_fitness - (best_fitness_history[0] if best_fitness_history else 0),
        'total_runtime_sec': total_time,
        'total_runtime_min': total_time / 60,
        'avg_time_per_gen': total_time / generation if generation > 0 else 0,
        'init_fitness_time': init_fitness_time,
        'target_reached': 'Yes' if best_overall_fitness >= target_fitness * max_fitness else 'No',
        'generations_to_best': best_fitness_history.index(best_overall_fitness) + 1 if best_overall_fitness in best_fitness_history else generation,
        'avg_final_fitness': avg_fitness_history[-1] if avg_fitness_history else 0,
        'C1_satisfied': constraint_stats['C1_satisfied'],
        'C2_satisfied': constraint_stats['C2_satisfied'],
        'C3_satisfied': constraint_stats['C3_satisfied'],
        'C4_satisfied': constraint_stats['C4_satisfied'],
        'perfect_groups': constraint_stats['perfect_groups'],
        'C1_pct': constraint_stats['C1_satisfied'] / K,
        'C2_pct': constraint_stats['C2_satisfied'] / K,
        'C3_pct': constraint_stats['C3_satisfied'] / K,
        'C4_pct': constraint_stats['C4_satisfied'] / K,
        'perfect_groups_pct': constraint_stats['perfect_groups'] / K,
        'best_fitness_history': best_fitness_history,
        'avg_fitness_history': avg_fitness_history
    }

print("✅ GA Runner Function defined")

✅ GA Runner Function defined


## 12. Define Test Scenarios

In [12]:
# Define 29 test scenarios based on research plan

# Skenario 1-10: Test popSize variations (10-100, step 10)
# Fixed: Generation=300, Cr=0.5, Mr=0.5
scenarios_popsize = []
for i, popsize in enumerate(range(10, 101, 10), start=1):
    scenarios_popsize.append({
        'scenario_id': i,
        'scenario_name': f'S{i:02d}_PopSize{popsize}',
        'phase': 'Phase 1: PopSize Test',
        'popsize': popsize,
        'generation': 300,
        'cr': 0.5,
        'mr': 0.5
    })

# Skenario 11-20: Test Generation variations (100-1000, step 100)
# Fixed: popSize=BEST_FROM_PHASE1 (placeholder, will be updated), Cr=0.5, Mr=0.5
scenarios_generation = []
BEST_POPSIZE_PLACEHOLDER = 50  # Will be determined from Phase 1 results
for i, generation in enumerate(range(100, 1001, 100), start=11):
    scenarios_generation.append({
        'scenario_id': i,
        'scenario_name': f'S{i:02d}_Gen{generation}',
        'phase': 'Phase 2: Generation Test',
        'popsize': BEST_POPSIZE_PLACEHOLDER,
        'generation': generation,
        'cr': 0.5,
        'mr': 0.5
    })

# Skenario 21-29: Test Cr:Mr combinations (0.1:0.9 to 0.9:0.1)
# Fixed: popSize=BEST_FROM_PHASE1, Generation=BEST_FROM_PHASE2
scenarios_crmr = []
BEST_GENERATION_PLACEHOLDER = 300  # Will be determined from Phase 2 results
for i, cr in enumerate(np.arange(0.1, 1.0, 0.1), start=21):
    mr = 1.0 - cr
    scenarios_crmr.append({
        'scenario_id': i,
        'scenario_name': f'S{i:02d}_Cr{cr:.1f}_Mr{mr:.1f}',
        'phase': 'Phase 3: Cr-Mr Test',
        'popsize': BEST_POPSIZE_PLACEHOLDER,
        'generation': BEST_GENERATION_PLACEHOLDER,
        'cr': round(cr, 1),
        'mr': round(mr, 1)
    })

# Combine all scenarios
all_scenarios = scenarios_popsize + scenarios_generation + scenarios_crmr

print("="*80)
print("📋 TEST SCENARIOS DEFINED")
print("="*80)
print(f"Total Scenarios: {len(all_scenarios)}")
print(f"  - Phase 1 (PopSize): {len(scenarios_popsize)} scenarios")
print(f"  - Phase 2 (Generation): {len(scenarios_generation)} scenarios")
print(f"  - Phase 3 (Cr-Mr): {len(scenarios_crmr)} scenarios")
print(f"\nEach scenario will be run 10 times with different seeds")
print(f"Total GA runs: {len(all_scenarios)} × 10 = {len(all_scenarios) * 10} runs")
print("="*80)

# Display sample scenarios
print("\n📊 Sample Scenarios:")
print(f"\nPhase 1 (first 3):")
for s in scenarios_popsize[:3]:
    print(f"  {s['scenario_name']}: PopSize={s['popsize']}, Gen={s['generation']}, Cr={s['cr']}, Mr={s['mr']}")

print(f"\nPhase 2 (first 3):")
for s in scenarios_generation[:3]:
    print(f"  {s['scenario_name']}: PopSize={s['popsize']}, Gen={s['generation']}, Cr={s['cr']}, Mr={s['mr']}")

print(f"\nPhase 3 (first 3):")
for s in scenarios_crmr[:3]:
    print(f"  {s['scenario_name']}: PopSize={s['popsize']}, Gen={s['generation']}, Cr={s['cr']}, Mr={s['mr']}")

print("\n⚠️  Note: Phase 2 & 3 use placeholder values for best PopSize and Generation.")
print("    Update these manually after analyzing Phase 1 and Phase 2 results.")

📋 TEST SCENARIOS DEFINED
Total Scenarios: 29
  - Phase 1 (PopSize): 10 scenarios
  - Phase 2 (Generation): 10 scenarios
  - Phase 3 (Cr-Mr): 9 scenarios

Each scenario will be run 10 times with different seeds
Total GA runs: 29 × 10 = 290 runs

📊 Sample Scenarios:

Phase 1 (first 3):
  S01_PopSize10: PopSize=10, Gen=300, Cr=0.5, Mr=0.5
  S02_PopSize20: PopSize=20, Gen=300, Cr=0.5, Mr=0.5
  S03_PopSize30: PopSize=30, Gen=300, Cr=0.5, Mr=0.5

Phase 2 (first 3):
  S11_Gen100: PopSize=50, Gen=100, Cr=0.5, Mr=0.5
  S12_Gen200: PopSize=50, Gen=200, Cr=0.5, Mr=0.5
  S13_Gen300: PopSize=50, Gen=300, Cr=0.5, Mr=0.5

Phase 3 (first 3):
  S21_Cr0.1_Mr0.9: PopSize=50, Gen=300, Cr=0.1, Mr=0.9
  S22_Cr0.2_Mr0.8: PopSize=50, Gen=300, Cr=0.2, Mr=0.8
  S23_Cr0.3_Mr0.7: PopSize=50, Gen=300, Cr=0.3, Mr=0.7

⚠️  Note: Phase 2 & 3 use placeholder values for best PopSize and Generation.
    Update these manually after analyzing Phase 1 and Phase 2 results.


## 13. Run All Test Scenarios

In [ ]:
# Configuration
NUM_RUNS_PER_SCENARIO = 10
STARTING_SEED = 42
TARGET_FITNESS = 1.0

# Storage for all results
all_results = []

print("="*80)
print("🚀 STARTING COMPREHENSIVE GA TESTING")
print("="*80)
print(f"Total Scenarios: {len(all_scenarios)}")
print(f"Runs per Scenario: {NUM_RUNS_PER_SCENARIO}")
print(f"Total Runs: {len(all_scenarios) * NUM_RUNS_PER_SCENARIO}")
print(f"Starting Seed: {STARTING_SEED}")
print("="*80)

# Overall timing
overall_start = time.time()

# Run all scenarios
for scenario_idx, scenario in enumerate(all_scenarios, start=1):
    scenario_id = scenario['scenario_id']
    scenario_name = scenario['scenario_name']
    phase = scenario['phase']
    popsize = scenario['popsize']
    generation = scenario['generation']
    cr = scenario['cr']
    mr = scenario['mr']
    
    print(f"\n{'='*80}")
    print(f"📍 SCENARIO {scenario_idx}/{len(all_scenarios)}: {scenario_name}")
    print(f"{'='*80}")
    print(f"Phase: {phase}")
    print(f"Parameters: PopSize={popsize}, Gen={generation}, Cr={cr}, Mr={mr}")
    print(f"Running {NUM_RUNS_PER_SCENARIO} independent runs...")
    
    scenario_start = time.time()
    scenario_results = []
    
    # Run 10 times with different seeds
    for run_id in range(1, NUM_RUNS_PER_SCENARIO + 1):
        seed = STARTING_SEED + (scenario_idx - 1) * NUM_RUNS_PER_SCENARIO + run_id
        
        print(f"\n  Run {run_id}/{NUM_RUNS_PER_SCENARIO} (Seed={seed})...", end=" ")
        run_start = time.time()
        
        try:
            result = run_ga_single(
                df_clean, preprocessed, 
                popsize, generation, cr, mr, 
                seed, TARGET_FITNESS
            )
            
            # Add scenario info to result
            result['scenario_id'] = scenario_id
            result['scenario_name'] = scenario_name
            result['phase'] = phase
            result['run_id'] = run_id
            result['popsize'] = popsize
            result['max_generation'] = generation
            result['cr'] = cr
            result['mr'] = mr
            
            scenario_results.append(result)
            all_results.append(result)
            
            run_time = time.time() - run_start
            print(f"✅ Done in {run_time:.1f}s | Fitness: {result['best_fitness']:.0f}/{preprocessed['max_fitness']} ({result['best_fitness_pct']:.2%}) | Gen: {result['final_generation']}")
            
        except Exception as e:
            print(f"❌ FAILED: {str(e)}")
            continue
    
    # Calculate scenario summary statistics
    if scenario_results:
        scenario_time = time.time() - scenario_start
        
        best_fitnesses = [r['best_fitness'] for r in scenario_results]
        runtimes = [r['total_runtime_sec'] for r in scenario_results]
        final_gens = [r['final_generation'] for r in scenario_results]
        
        print(f"\n{'─'*80}")
        print(f"📊 SCENARIO {scenario_name} SUMMARY:")
        print(f"{'─'*80}")
        print(f"  Best Fitness (mean ± std): {np.mean(best_fitnesses):.2f} ± {np.std(best_fitnesses):.2f}")
        print(f"  Best Fitness (min-max): {np.min(best_fitnesses):.2f} - {np.max(best_fitnesses):.2f}")
        print(f"  Runtime (mean ± std): {np.mean(runtimes):.1f}s ± {np.std(runtimes):.1f}s")
        print(f"  Final Generation (mean): {np.mean(final_gens):.1f}")
        print(f"  Scenario Total Time: {scenario_time/60:.2f} minutes")
        
        elapsed_total = time.time() - overall_start
        remaining_scenarios = len(all_scenarios) - scenario_idx
        avg_time_per_scenario = elapsed_total / scenario_idx
        eta_seconds = remaining_scenarios * avg_time_per_scenario
        eta_hours = eta_seconds / 3600
        
        print(f"\n⏱️  Progress: {scenario_idx}/{len(all_scenarios)} scenarios completed")
        print(f"  Elapsed Time: {elapsed_total/3600:.2f} hours")
        print(f"  ETA: {eta_hours:.2f} hours")
        print(f"{'─'*80}")

print(f"\n{'='*80}")
print("✅ ALL SCENARIOS COMPLETED")
print(f"{'='*80}")
total_time = time.time() - overall_start
print(f"Total Runs: {len(all_results)}")
print(f"Total Time: {total_time/3600:.2f} hours")
print(f"Average Time per Run: {total_time/len(all_results):.1f} seconds")
print(f"{'='*80}")

# Validate parameters
assert POPSIZE >= 2, "POPSIZE must be at least 2 for crossover to work!"
assert 0 <= CR <= 1, "CR (Crossover Rate) must be between 0 and 1!"
assert 0 <= MR <= 1, "MR (Mutation Rate) must be between 0 and 1!"

# Extract preprocessed data
N = preprocessed['N']
K = preprocessed['K']
PL = preprocessed['PL']
PP = preprocessed['PP']
expected_sizes = preprocessed['expected_sizes']
max_fitness = preprocessed['max_fitness']

print("="*80)
print("🚀 STARTING GENETIC ALGORITHM (OPTIMIZED)")
print("="*80)
print(f"Population Size: {POPSIZE}")
print(f"Crossover Rate: {CR}")
print(f"Mutation Rate: {MR}")
print(f"Max Generation: {MAX_GENERATION}")
print(f"Target Fitness: {TARGET_FITNESS} (={TARGET_FITNESS * max_fitness:.0f}/{max_fitness})")
print("="*80)

# Initialize
start_time = time.time()
population = initialize_population(df_clean, POPSIZE)
print(f"✅ Initial population created ({POPSIZE} individuals)")

# 🔥 NEW: Calculate initial fitness ONCE and cache it
print(f"🔍 Calculating initial fitness (one-time cost)...")
init_fitness_start = time.time()
population_fitness = []
for i, kromosom in enumerate(population):
    fitness = calculate_fitness(kromosom, df_clean, expected_sizes, PL, PP)
    population_fitness.append(fitness)
    if (i + 1) % 5 == 0 or (i + 1) == POPSIZE:
        print(f"   Progress: {i+1}/{POPSIZE} individuals evaluated")

init_fitness_time = time.time() - init_fitness_start
print(f"✅ Initial fitness calculated in {init_fitness_time:.1f}s")
print("="*80)

# Track best solution
best_fitness_history = []
avg_fitness_history = []
best_overall_fitness = max(population_fitness)
best_overall_solution = population[population_fitness.index(best_overall_fitness)].copy()

print(f"📊 Initial Best Fitness: {best_overall_fitness:.0f}/{max_fitness} ({best_overall_fitness/max_fitness:.2%})")
print("="*80)

# Main GA Loop
for generation in range(1, MAX_GENERATION + 1):
    gen_start = time.time()
    
    # Crossover
    cx_start = time.time()
    parent_pairs = select_parents_for_crossover(population, CR)
    offspring_cx = []
    for p1, p2 in parent_pairs:
        c1, c2 = pmx_crossover(p1, p2)
        offspring_cx.extend([c1, c2])
    cx_time = time.time() - cx_start
    
    # Mutation
    mut_start = time.time()
    parents_mut = select_parents_for_mutation(population, MR)
    offspring_mut = [reciprocal_exchange_mutation(p) for p in parents_mut]
    mut_time = time.time() - mut_start
    
    # Combine offspring
    offspring = offspring_cx + offspring_mut
    
    # 🔥 OPTIMIZED: Only calculate fitness for new offspring
    repl_start = time.time()
    population, population_fitness = elitism_replacement_optimized(
        population, population_fitness, offspring, 
        df_clean, expected_sizes, PL, PP, POPSIZE
    )
    repl_time = time.time() - repl_start
    
    # Track statistics
    best_fitness = population_fitness[0]
    avg_fitness = np.mean(population_fitness)
    best_fitness_history.append(best_fitness)
    avg_fitness_history.append(avg_fitness)
    
    # Update best overall
    if best_fitness > best_overall_fitness:
        best_overall_fitness = best_fitness
        best_overall_solution = population[0].copy()
        print(f"🆕 NEW BEST at Gen {generation}: {best_fitness:.0f}/{max_fitness} ({best_fitness/max_fitness:.2%})")
    
    # Progress every 5 generations
    if generation % 5 == 0 or generation == 1:
        elapsed = time.time() - start_time
        gen_time = time.time() - gen_start
        num_offspring = len(offspring)
        print(f"Gen {generation:3d}/{MAX_GENERATION} | Best: {best_fitness:3.0f}/{max_fitness} ({best_fitness/max_fitness:.2%}) | "
              f"Avg: {avg_fitness:6.2f} | Offspring: {num_offspring:2d} | "
              f"Time: {gen_time:.1f}s (CX:{cx_time:.2f}s MUT:{mut_time:.2f}s REPL:{repl_time:.1f}s) | "
              f"Total: {elapsed/60:.1f}m")
    
    # Check termination
    if best_fitness >= TARGET_FITNESS * max_fitness:
        print(f"🎯 TARGET REACHED at Generation {generation}!")
        break

# Final results
total_time = time.time() - start_time
print("="*80)
print("✅ GA COMPLETE")
print("="*80)
print(f"Best Fitness: {best_overall_fitness:.0f}/{max_fitness} ({best_overall_fitness/max_fitness:.2%})")
print(f"Final Generation: {generation}/{MAX_GENERATION}")
print(f"Total Time: {total_time/60:.2f} minutes ({total_time:.1f} seconds)")
print(f"Avg Time per Generation: {total_time/generation:.2f} seconds")
print("="*80)

🚀 STARTING COMPREHENSIVE GA TESTING
Total Scenarios: 29
Runs per Scenario: 10
Total Runs: 290
Starting Seed: 42

📍 SCENARIO 1/29: S01_PopSize10
Phase: Phase 1: PopSize Test
Parameters: PopSize=10, Gen=300, Cr=0.5, Mr=0.5
Running 10 independent runs...

  Run 1/10 (Seed=43)... ✅ Done in 483.0s | Fitness: 711/760 (93.55%) | Gen: 300

  Run 2/10 (Seed=44)... ✅ Done in 483.0s | Fitness: 711/760 (93.55%) | Gen: 300

  Run 2/10 (Seed=44)... ✅ Done in 329.5s | Fitness: 704/760 (92.63%) | Gen: 300

  Run 3/10 (Seed=45)... ✅ Done in 329.5s | Fitness: 704/760 (92.63%) | Gen: 300

  Run 3/10 (Seed=45)... ✅ Done in 325.3s | Fitness: 699/760 (91.97%) | Gen: 300

  Run 4/10 (Seed=46)... ✅ Done in 325.3s | Fitness: 699/760 (91.97%) | Gen: 300

  Run 4/10 (Seed=46)... ✅ Done in 327.8s | Fitness: 708/760 (93.16%) | Gen: 300

  Run 5/10 (Seed=47)... ✅ Done in 327.8s | Fitness: 708/760 (93.16%) | Gen: 300

  Run 5/10 (Seed=47)... ✅ Done in 411.7s | Fitness: 714/760 (93.95%) | Gen: 300

  Run 6/10 (Seed=4

## 14. Export All Results to CSV

In [ ]:
# Export detailed results for each run
print("="*80)
print("💾 EXPORTING RESULTS")
print("="*80)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 1. Export per-run detailed results
print("\n1️⃣ Exporting per-run detailed results...")
detailed_results = []
for result in all_results:
    detailed_results.append({
        'Scenario_ID': result['scenario_id'],
        'Scenario_Name': result['scenario_name'],
        'Phase': result['phase'],
        'Run_ID': result['run_id'],
        'Seed': result['seed'],
        'PopSize': result['popsize'],
        'Max_Generation': result['max_generation'],
        'Cr': result['cr'],
        'Mr': result['mr'],
        'Best_Fitness': result['best_fitness'],
        'Best_Fitness_Pct': result['best_fitness_pct'],
        'Final_Generation': result['final_generation'],
        'Initial_Best_Fitness': result['initial_best_fitness'],
        'Fitness_Improvement': result['fitness_improvement'],
        'Total_Runtime_Sec': result['total_runtime_sec'],
        'Total_Runtime_Min': result['total_runtime_min'],
        'Avg_Time_Per_Gen': result['avg_time_per_gen'],
        'Init_Fitness_Time': result['init_fitness_time'],
        'Target_Reached': result['target_reached'],
        'Generations_To_Best': result['generations_to_best'],
        'Avg_Final_Fitness': result['avg_final_fitness'],
        'C1_Satisfied': result['C1_satisfied'],
        'C2_Satisfied': result['C2_satisfied'],
        'C3_Satisfied': result['C3_satisfied'],
        'C4_Satisfied': result['C4_satisfied'],
        'Perfect_Groups': result['perfect_groups'],
        'C1_Pct': result['C1_pct'],
        'C2_Pct': result['C2_pct'],
        'C3_Pct': result['C3_pct'],
        'C4_Pct': result['C4_pct'],
        'Perfect_Groups_Pct': result['perfect_groups_pct']
    })

detailed_df = pd.DataFrame(detailed_results)
detailed_file = f'{OUTPUT_DIR}/skenario_results/all_runs_detailed_{timestamp}.csv'
detailed_df.to_csv(detailed_file, index=False)
print(f"   ✅ Saved: {detailed_file}")
print(f"      Rows: {len(detailed_df)}")

# 2. Export aggregated summary per scenario
print("\n2️⃣ Exporting aggregated summary per scenario...")
summary_results = []

for scenario in all_scenarios:
    scenario_name = scenario['scenario_name']
    scenario_data = [r for r in all_results if r['scenario_name'] == scenario_name]
    
    if not scenario_data:
        continue
    
    best_fitnesses = [r['best_fitness'] for r in scenario_data]
    best_fitness_pcts = [r['best_fitness_pct'] for r in scenario_data]
    runtimes = [r['total_runtime_sec'] for r in scenario_data]
    final_gens = [r['final_generation'] for r in scenario_data]
    gens_to_best = [r['generations_to_best'] for r in scenario_data]
    perfect_groups = [r['perfect_groups'] for r in scenario_data]
    perfect_groups_pcts = [r['perfect_groups_pct'] for r in scenario_data]
    
    summary_results.append({
        'Scenario_ID': scenario['scenario_id'],
        'Scenario_Name': scenario_name,
        'Phase': scenario['phase'],
        'PopSize': scenario['popsize'],
        'Max_Generation': scenario['generation'],
        'Cr': scenario['cr'],
        'Mr': scenario['mr'],
        'Num_Runs': len(scenario_data),
        
        # Best Fitness Statistics
        'Best_Fitness_Mean': np.mean(best_fitnesses),
        'Best_Fitness_Std': np.std(best_fitnesses),
        'Best_Fitness_Min': np.min(best_fitnesses),
        'Best_Fitness_Max': np.max(best_fitnesses),
        'Best_Fitness_Median': np.median(best_fitnesses),
        'Best_Fitness_Pct_Mean': np.mean(best_fitness_pcts),
        'Best_Fitness_Pct_Std': np.std(best_fitness_pcts),
        
        # Runtime Statistics
        'Runtime_Mean_Sec': np.mean(runtimes),
        'Runtime_Std_Sec': np.std(runtimes),
        'Runtime_Min_Sec': np.min(runtimes),
        'Runtime_Max_Sec': np.max(runtimes),
        'Runtime_Mean_Min': np.mean(runtimes) / 60,
        
        # Generation Statistics
        'Final_Gen_Mean': np.mean(final_gens),
        'Final_Gen_Std': np.std(final_gens),
        'Gens_To_Best_Mean': np.mean(gens_to_best),
        'Gens_To_Best_Std': np.std(gens_to_best),
        
        # Constraint Satisfaction
        'Perfect_Groups_Mean': np.mean(perfect_groups),
        'Perfect_Groups_Std': np.std(perfect_groups),
        'Perfect_Groups_Pct_Mean': np.mean(perfect_groups_pcts),
        'Perfect_Groups_Pct_Std': np.std(perfect_groups_pcts),
        
        # Success Rate
        'Target_Reached_Count': sum(1 for r in scenario_data if r['target_reached'] == 'Yes'),
        'Target_Reached_Pct': sum(1 for r in scenario_data if r['target_reached'] == 'Yes') / len(scenario_data)
    })

summary_df = pd.DataFrame(summary_results)
summary_file = f'{OUTPUT_DIR}/summary/scenarios_summary_{timestamp}.csv'
summary_df.to_csv(summary_file, index=False)
print(f"   ✅ Saved: {summary_file}")
print(f"      Rows: {len(summary_df)}")

# 3. Export best configurations per phase
print("\n3️⃣ Exporting best configurations per phase...")
best_configs = []

for phase in ['Phase 1: PopSize Test', 'Phase 2: Generation Test', 'Phase 3: Cr-Mr Test']:
    phase_data = summary_df[summary_df['Phase'] == phase]
    if len(phase_data) > 0:
        best_row = phase_data.loc[phase_data['Best_Fitness_Mean'].idxmax()]
        best_configs.append({
            'Phase': phase,
            'Best_Scenario': best_row['Scenario_Name'],
            'PopSize': best_row['PopSize'],
            'Generation': best_row['Max_Generation'],
            'Cr': best_row['Cr'],
            'Mr': best_row['Mr'],
            'Best_Fitness_Mean': best_row['Best_Fitness_Mean'],
            'Best_Fitness_Std': best_row['Best_Fitness_Std'],
            'Runtime_Mean_Min': best_row['Runtime_Mean_Min'],
            'Perfect_Groups_Pct_Mean': best_row['Perfect_Groups_Pct_Mean']
        })

best_configs_df = pd.DataFrame(best_configs)
best_configs_file = f'{OUTPUT_DIR}/summary/best_configurations_{timestamp}.csv'
best_configs_df.to_csv(best_configs_file, index=False)
print(f"   ✅ Saved: {best_configs_file}")
print(f"      Rows: {len(best_configs_df)}")

print("\n" + "="*80)
print("✅ ALL RESULTS EXPORTED")
print("="*80)
print(f"Files saved:")
print(f"  1. {detailed_file}")
print(f"  2. {summary_file}")
print(f"  3. {best_configs_file}")
print("="*80)

# Display summary statistics
print("\n📊 QUICK SUMMARY:")
print(f"\nTop 5 Scenarios by Mean Best Fitness:")
top5 = summary_df.nlargest(5, 'Best_Fitness_Mean')[['Scenario_Name', 'PopSize', 'Max_Generation', 'Cr', 'Mr', 'Best_Fitness_Mean', 'Best_Fitness_Std']]
print(top5.to_string(index=False))

print(f"\n\nBest Configuration per Phase:")
print(best_configs_df.to_string(index=False))

## 15. Visualize Comparison Plots

In [ ]:
# Create comprehensive comparison plots
print("="*80)
print("📊 GENERATING COMPARISON PLOTS")
print("="*80)

# 1. Phase 1: PopSize comparison
print("\n1️⃣ Creating PopSize comparison plot...")
phase1_data = summary_df[summary_df['Phase'] == 'Phase 1: PopSize Test'].sort_values('PopSize')

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Phase 1: Population Size Comparison', fontsize=16, fontweight='bold')

# Plot 1: Best Fitness vs PopSize
axes[0, 0].errorbar(phase1_data['PopSize'], phase1_data['Best_Fitness_Mean'], 
                     yerr=phase1_data['Best_Fitness_Std'], marker='o', capsize=5, linewidth=2)
axes[0, 0].set_xlabel('Population Size', fontsize=12)
axes[0, 0].set_ylabel('Best Fitness (Mean ± Std)', fontsize=12)
axes[0, 0].set_title('Best Fitness vs Population Size', fontsize=13, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=preprocessed['max_fitness'], color='r', linestyle='--', alpha=0.5, label='Max Fitness')
axes[0, 0].legend()

# Plot 2: Runtime vs PopSize
axes[0, 1].plot(phase1_data['PopSize'], phase1_data['Runtime_Mean_Min'], marker='s', linewidth=2, color='orange')
axes[0, 1].fill_between(phase1_data['PopSize'], 
                         phase1_data['Runtime_Mean_Min'] - phase1_data['Runtime_Std_Sec']/60,
                         phase1_data['Runtime_Mean_Min'] + phase1_data['Runtime_Std_Sec']/60,
                         alpha=0.3, color='orange')
axes[0, 1].set_xlabel('Population Size', fontsize=12)
axes[0, 1].set_ylabel('Runtime (minutes)', fontsize=12)
axes[0, 1].set_title('Runtime vs Population Size', fontsize=13, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Perfect Groups vs PopSize
axes[1, 0].plot(phase1_data['PopSize'], phase1_data['Perfect_Groups_Pct_Mean']*100, marker='^', linewidth=2, color='green')
axes[1, 0].set_xlabel('Population Size', fontsize=12)
axes[1, 0].set_ylabel('Perfect Groups (%)', fontsize=12)
axes[1, 0].set_title('Perfect Groups Percentage vs Population Size', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_ylim([0, 105])

# Plot 4: Final Generation vs PopSize
axes[1, 1].plot(phase1_data['PopSize'], phase1_data['Final_Gen_Mean'], marker='d', linewidth=2, color='purple')
axes[1, 1].fill_between(phase1_data['PopSize'],
                         phase1_data['Final_Gen_Mean'] - phase1_data['Final_Gen_Std'],
                         phase1_data['Final_Gen_Mean'] + phase1_data['Final_Gen_Std'],
                         alpha=0.3, color='purple')
axes[1, 1].set_xlabel('Population Size', fontsize=12)
axes[1, 1].set_ylabel('Final Generation (Mean ± Std)', fontsize=12)
axes[1, 1].set_title('Convergence Generation vs Population Size', fontsize=13, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
phase1_plot = f'{OUTPUT_DIR}/plots/phase1_popsize_comparison_{timestamp}.png'
plt.savefig(phase1_plot, dpi=300, bbox_inches='tight')
print(f"   ✅ Saved: {phase1_plot}")
plt.show()

# 2. Phase 2: Generation comparison
print("\n2️⃣ Creating Generation comparison plot...")
phase2_data = summary_df[summary_df['Phase'] == 'Phase 2: Generation Test'].sort_values('Max_Generation')

if len(phase2_data) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Phase 2: Max Generation Comparison', fontsize=16, fontweight='bold')
    
    # Plot 1: Best Fitness vs Generation
    axes[0, 0].errorbar(phase2_data['Max_Generation'], phase2_data['Best_Fitness_Mean'], 
                         yerr=phase2_data['Best_Fitness_Std'], marker='o', capsize=5, linewidth=2)
    axes[0, 0].set_xlabel('Max Generation', fontsize=12)
    axes[0, 0].set_ylabel('Best Fitness (Mean ± Std)', fontsize=12)
    axes[0, 0].set_title('Best Fitness vs Max Generation', fontsize=13, fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].axhline(y=preprocessed['max_fitness'], color='r', linestyle='--', alpha=0.5, label='Max Fitness')
    axes[0, 0].legend()
    
    # Plot 2: Runtime vs Generation
    axes[0, 1].plot(phase2_data['Max_Generation'], phase2_data['Runtime_Mean_Min'], marker='s', linewidth=2, color='orange')
    axes[0, 1].set_xlabel('Max Generation', fontsize=12)
    axes[0, 1].set_ylabel('Runtime (minutes)', fontsize=12)
    axes[0, 1].set_title('Runtime vs Max Generation', fontsize=13, fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Perfect Groups vs Generation
    axes[1, 0].plot(phase2_data['Max_Generation'], phase2_data['Perfect_Groups_Pct_Mean']*100, marker='^', linewidth=2, color='green')
    axes[1, 0].set_xlabel('Max Generation', fontsize=12)
    axes[1, 0].set_ylabel('Perfect Groups (%)', fontsize=12)
    axes[1, 0].set_title('Perfect Groups Percentage vs Max Generation', fontsize=13, fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].set_ylim([0, 105])
    
    # Plot 4: Convergence Speed
    axes[1, 1].plot(phase2_data['Max_Generation'], phase2_data['Gens_To_Best_Mean'], marker='d', linewidth=2, color='purple')
    axes[1, 1].set_xlabel('Max Generation', fontsize=12)
    axes[1, 1].set_ylabel('Generations to Best', fontsize=12)
    axes[1, 1].set_title('Convergence Speed vs Max Generation', fontsize=13, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    phase2_plot = f'{OUTPUT_DIR}/plots/phase2_generation_comparison_{timestamp}.png'
    plt.savefig(phase2_plot, dpi=300, bbox_inches='tight')
    print(f"   ✅ Saved: {phase2_plot}")
    plt.show()

# 3. Phase 3: Cr-Mr combination comparison
print("\n3️⃣ Creating Cr-Mr combination comparison plot...")
phase3_data = summary_df[summary_df['Phase'] == 'Phase 3: Cr-Mr Test'].sort_values('Cr')

if len(phase3_data) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Phase 3: Crossover-Mutation Rate Comparison', fontsize=16, fontweight='bold')
    
    x_labels = [f"{cr:.1f}:{mr:.1f}" for cr, mr in zip(phase3_data['Cr'], phase3_data['Mr'])]
    x_pos = np.arange(len(x_labels))
    
    # Plot 1: Best Fitness vs Cr:Mr
    axes[0, 0].bar(x_pos, phase3_data['Best_Fitness_Mean'], yerr=phase3_data['Best_Fitness_Std'], 
                   capsize=5, alpha=0.7, color='steelblue')
    axes[0, 0].set_xticks(x_pos)
    axes[0, 0].set_xticklabels(x_labels, rotation=45, ha='right')
    axes[0, 0].set_xlabel('Cr:Mr Ratio', fontsize=12)
    axes[0, 0].set_ylabel('Best Fitness (Mean ± Std)', fontsize=12)
    axes[0, 0].set_title('Best Fitness vs Cr:Mr Ratio', fontsize=13, fontweight='bold')
    axes[0, 0].axhline(y=preprocessed['max_fitness'], color='r', linestyle='--', alpha=0.5, label='Max Fitness')
    axes[0, 0].grid(True, alpha=0.3, axis='y')
    axes[0, 0].legend()
    
    # Plot 2: Runtime vs Cr:Mr
    axes[0, 1].plot(x_pos, phase3_data['Runtime_Mean_Min'], marker='o', linewidth=2, color='orange')
    axes[0, 1].set_xticks(x_pos)
    axes[0, 1].set_xticklabels(x_labels, rotation=45, ha='right')
    axes[0, 1].set_xlabel('Cr:Mr Ratio', fontsize=12)
    axes[0, 1].set_ylabel('Runtime (minutes)', fontsize=12)
    axes[0, 1].set_title('Runtime vs Cr:Mr Ratio', fontsize=13, fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Perfect Groups vs Cr:Mr
    axes[1, 0].bar(x_pos, phase3_data['Perfect_Groups_Pct_Mean']*100, alpha=0.7, color='green')
    axes[1, 0].set_xticks(x_pos)
    axes[1, 0].set_xticklabels(x_labels, rotation=45, ha='right')
    axes[1, 0].set_xlabel('Cr:Mr Ratio', fontsize=12)
    axes[1, 0].set_ylabel('Perfect Groups (%)', fontsize=12)
    axes[1, 0].set_title('Perfect Groups Percentage vs Cr:Mr Ratio', fontsize=13, fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    axes[1, 0].set_ylim([0, 105])
    
    # Plot 4: Target Reached vs Cr:Mr
    axes[1, 1].bar(x_pos, phase3_data['Target_Reached_Pct']*100, alpha=0.7, color='purple')
    axes[1, 1].set_xticks(x_pos)
    axes[1, 1].set_xticklabels(x_labels, rotation=45, ha='right')
    axes[1, 1].set_xlabel('Cr:Mr Ratio', fontsize=12)
    axes[1, 1].set_ylabel('Target Reached (%)', fontsize=12)
    axes[1, 1].set_title('Success Rate vs Cr:Mr Ratio', fontsize=13, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    axes[1, 1].set_ylim([0, 105])
    
    plt.tight_layout()
    phase3_plot = f'{OUTPUT_DIR}/plots/phase3_crmr_comparison_{timestamp}.png'
    plt.savefig(phase3_plot, dpi=300, bbox_inches='tight')
    print(f"   ✅ Saved: {phase3_plot}")
    plt.show()

# 4. Overall summary heatmap
print("\n4️⃣ Creating overall summary heatmap...")
fig, ax = plt.subplots(1, 1, figsize=(14, 10))

# Prepare data for heatmap
heatmap_data = summary_df[['Scenario_Name', 'Best_Fitness_Pct_Mean', 'Runtime_Mean_Min', 'Perfect_Groups_Pct_Mean']].copy()
heatmap_data['Best_Fitness_Pct_Mean'] *= 100
heatmap_data['Perfect_Groups_Pct_Mean'] *= 100
heatmap_data = heatmap_data.set_index('Scenario_Name')

# Normalize for better visualization
from matplotlib.colors import Normalize
norm = Normalize(vmin=0, vmax=100)

im = ax.imshow(heatmap_data.T, aspect='auto', cmap='RdYlGn', norm=norm)

# Set ticks
ax.set_xticks(np.arange(len(heatmap_data.index)))
ax.set_yticks(np.arange(len(heatmap_data.columns)))
ax.set_xticklabels(heatmap_data.index, rotation=90, ha='right', fontsize=8)
ax.set_yticklabels(['Best Fitness %', 'Runtime (min)', 'Perfect Groups %'], fontsize=11)

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Value', fontsize=12)

# Add title
ax.set_title('Scenario Comparison Heatmap', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
heatmap_plot = f'{OUTPUT_DIR}/plots/overall_heatmap_{timestamp}.png'
plt.savefig(heatmap_plot, dpi=300, bbox_inches='tight')
print(f"   ✅ Saved: {heatmap_plot}")
plt.show()

print("\n" + "="*80)
print("✅ ALL PLOTS GENERATED")
print("="*80)

## 16. Test Execution Instructions

### 📌 How to Use This Notebook

**Step 1: Review Scenarios**
- Cell 12 defines all 29 test scenarios
- **IMPORTANT**: Update `BEST_POPSIZE_PLACEHOLDER` and `BEST_GENERATION_PLACEHOLDER` after completing Phase 1 and Phase 2

**Step 2: Run Testing**
- Execute Cell 13 to run all 290 GA experiments (29 scenarios × 10 runs)
- **Estimated time**: 10-50 hours depending on hardware
- Progress is displayed every 5 generations and after each scenario

**Step 3: Analyze Results**
- Cell 14 exports 3 CSV files with detailed and summarized results
- Cell 15 generates 4 comprehensive comparison plots
- Review `best_configurations_{timestamp}.csv` to find optimal parameters

**Step 4: Iterative Optimization**
1. Run Phase 1 (Scenarios 1-10) to find best PopSize
2. Update `BEST_POPSIZE_PLACEHOLDER` in Cell 12
3. Run Phase 2 (Scenarios 11-20) to find best Generation
4. Update `BEST_GENERATION_PLACEHOLDER` in Cell 12
5. Run Phase 3 (Scenarios 21-29) to find best Cr:Mr combination

### 📂 Output Structure
```
output/
├── skenario_results/
│   └── all_runs_detailed_{timestamp}.csv (290 rows)
├── summary/
│   ├── scenarios_summary_{timestamp}.csv (29 rows)
│   └── best_configurations_{timestamp}.csv (3 rows)
└── plots/
    ├── phase1_popsize_comparison_{timestamp}.png
    ├── phase2_generation_comparison_{timestamp}.png
    ├── phase3_crmr_comparison_{timestamp}.png
    └── overall_heatmap_{timestamp}.png
```

### ⚠️ Important Notes
- Each run uses a different random seed for reproducibility
- Seeds are based on: `STARTING_SEED + (scenario_idx - 1) * 10 + run_id`
- Default starting seed is 42
- All fitness values are maximized (higher = better)
- Runtime includes initialization, evolution, and fitness evaluation

### 🎯 Success Criteria
- **Target Fitness**: 100% (all 760 constraints satisfied = 190 groups × 4 constraints)
- **Best Fitness**: Maximum fitness achieved across all runs
- **Perfect Groups**: Number of groups satisfying all 4 constraints (C1, C2, C3, C4)

## 12. Visualize Convergence

In [ ]:
plt.figure(figsize=(12, 5))

# Best Fitness
plt.subplot(1, 2, 1)
plt.plot(range(1, len(best_fitness_history) + 1), best_fitness_history, 'b-', linewidth=2, label='Best Fitness')
plt.axhline(y=max_fitness, color='r', linestyle='--', label=f'Max Fitness ({max_fitness})')
plt.axhline(y=TARGET_FITNESS * max_fitness, color='g', linestyle='--', label=f'Target ({TARGET_FITNESS * max_fitness:.0f})')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Best Fitness Convergence')
plt.legend()
plt.grid(True, alpha=0.3)

# Average Fitness
plt.subplot(1, 2, 2)
plt.plot(range(1, len(avg_fitness_history) + 1), avg_fitness_history, 'orange', linewidth=2, label='Avg Fitness')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Average Fitness Convergence')
plt.legend()
plt.grid(True, alpha=0.3)

# Generate timestamp for unique filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

plt.tight_layout()
plot_filename = f'{OUTPUT_DIR}/plot/convergence_plot_{timestamp}.png'
plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Convergence plot saved to {plot_filename}")

## 13. Export Best Solution to Excel

In [ ]:
# Generate timestamp for unique filename (reuse from previous cell if exists)
if 'timestamp' not in locals():
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Decode best solution
best_groups = decode_kromosom(best_overall_solution, df_clean, expected_sizes)

# Prepare export data
export_data = []
for i, group_df in enumerate(best_groups):
    group_num = i + 1
    
    # Calculate constraints
    c1 = evaluate_C1(group_df)
    c2 = evaluate_C2(group_df)
    c3 = evaluate_C3(group_df, PL, PP)
    c4 = evaluate_C4(group_df, expected_sizes[i])
    group_fitness = c1 + c2 + c3 + c4
    
    # Add each student
    for _, student in group_df.iterrows():
        export_data.append({
            'Kelompok': group_num,
            'ID': student['ID'],
            'Jenis Kelamin': student['Jenis Kelamin'],
            'Jurusan': student['Jurusan'],
            'HTQ': student['HTQ'],
            'C1': c1,
            'C2': c2,
            'C3': c3,
            'C4': c4,
            'Group Fitness': group_fitness
        })

# Create DataFrame and export with timestamp as CSV
result_df = pd.DataFrame(export_data)
hasil_file = f'{OUTPUT_DIR}/hasil/best_solution_kelompok_kkm_{timestamp}.csv'
result_df.to_csv(hasil_file, index=False)

print("="*80)
print("✅ GROUPING RESULT EXPORTED")
print("="*80)
print(f"File saved: {hasil_file}")
print(f"Total rows: {len(result_df)}")
print(f"Total groups: {K}")
print(f"Best fitness: {best_overall_fitness:.0f}/{max_fitness} ({best_overall_fitness/max_fitness:.2%})")
print(f"Timestamp: {timestamp}")
print("="*80)

# Show sample
print("\n📊 Sample output (first 10 rows):")
result_df.head(10)

## 14. Export GA Statistics

In [ ]:
# Reuse timestamp from previous cells
if 'timestamp' not in locals():
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Prepare GA statistics
ga_stats = {
    'Timestamp': timestamp,
    'Run Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    
    # Dataset Info
    'Total Students': N,
    'Number of Groups': K,
    'Male Students': preprocessed['L'],
    'Female Students': preprocessed['P'],
    'Male Proportion': f"{preprocessed['PL']:.2%}",
    'Female Proportion': f"{preprocessed['PP']:.2%}",
    'Unique Majors': df_clean['Jurusan'].nunique(),
    'HTQ Members': df_clean['HTQ'].sum(),
    
    # GA Parameters
    'Population Size': POPSIZE,
    'Crossover Rate': CR,
    'Mutation Rate': MR,
    'Max Generation': MAX_GENERATION,
    'Target Fitness': TARGET_FITNESS,
    'Max Possible Fitness': max_fitness,
    
    # Results
    'Final Generation': generation,
    'Best Fitness': best_overall_fitness,
    'Best Fitness Percentage': f"{best_overall_fitness/max_fitness:.2%}",
    'Average Final Fitness': f"{avg_fitness:.2f}",
    'Initial Best Fitness': best_fitness_history[0] if best_fitness_history else 0,
    'Fitness Improvement': best_overall_fitness - (best_fitness_history[0] if best_fitness_history else 0),
    
    # Performance
    'Total Runtime (seconds)': f"{total_time:.2f}",
    'Total Runtime (minutes)': f"{total_time/60:.2f}",
    'Avg Time per Generation (seconds)': f"{total_time/generation:.2f}",
    'Initial Fitness Calculation Time (seconds)': f"{init_fitness_time:.2f}",
    
    # Convergence
    'Generations to Best': best_fitness_history.index(best_overall_fitness) + 1 if best_overall_fitness in best_fitness_history else generation,
    'Target Reached': 'Yes' if best_overall_fitness >= TARGET_FITNESS * max_fitness else 'No',
    'Convergence Rate': f"{(best_overall_fitness - best_fitness_history[0]) / generation:.2f}" if generation > 0 and best_fitness_history else 0,
}

# Calculate constraint satisfaction statistics
constraint_stats = {
    'C1_satisfied': 0,  # HTQ >= 1
    'C2_satisfied': 0,  # Jurusan > 50%
    'C3_satisfied': 0,  # Gender ±10%
    'C4_satisfied': 0,  # Exact size
    'perfect_groups': 0  # All 4 constraints satisfied
}

for i, group_df in enumerate(best_groups):
    c1 = evaluate_C1(group_df)
    c2 = evaluate_C2(group_df)
    c3 = evaluate_C3(group_df, PL, PP)
    c4 = evaluate_C4(group_df, expected_sizes[i])
    
    constraint_stats['C1_satisfied'] += c1
    constraint_stats['C2_satisfied'] += c2
    constraint_stats['C3_satisfied'] += c3
    constraint_stats['C4_satisfied'] += c4
    
    if c1 + c2 + c3 + c4 == 4:
        constraint_stats['perfect_groups'] += 1

# Add constraint stats to GA stats
ga_stats.update({
    'Groups with HTQ (C1)': constraint_stats['C1_satisfied'],
    'Groups with HTQ %': f"{constraint_stats['C1_satisfied']/K:.2%}",
    'Groups with Diverse Majors (C2)': constraint_stats['C2_satisfied'],
    'Groups with Diverse Majors %': f"{constraint_stats['C2_satisfied']/K:.2%}",
    'Groups with Balanced Gender (C3)': constraint_stats['C3_satisfied'],
    'Groups with Balanced Gender %': f"{constraint_stats['C3_satisfied']/K:.2%}",
    'Groups with Correct Size (C4)': constraint_stats['C4_satisfied'],
    'Groups with Correct Size %': f"{constraint_stats['C4_satisfied']/K:.2%}",
    'Perfect Groups (All Constraints)': constraint_stats['perfect_groups'],
    'Perfect Groups %': f"{constraint_stats['perfect_groups']/K:.2%}",
})

# Export to CSV
stats_df = pd.DataFrame([ga_stats]).T
stats_df.columns = ['Value']
stats_file = f'{OUTPUT_DIR}/statistik/ga_statistics_{timestamp}.csv'
stats_df.to_csv(stats_file, header=True)

# Also export fitness history
history_df = pd.DataFrame({
    'Generation': range(1, len(best_fitness_history) + 1),
    'Best_Fitness': best_fitness_history,
    'Avg_Fitness': avg_fitness_history,
    'Best_Fitness_Percentage': [f/max_fitness for f in best_fitness_history]
})
history_file = f'{OUTPUT_DIR}/statistik/fitness_history_{timestamp}.csv'
history_df.to_csv(history_file, index=False)

print("="*80)
print("✅ GA STATISTICS EXPORTED")
print("="*80)
print(f"Statistics file: {stats_file}")
print(f"Fitness history: {history_file}")
print("="*80)
print("\n📊 Key Statistics:")
print(f"   Best Fitness: {best_overall_fitness:.0f}/{max_fitness} ({best_overall_fitness/max_fitness:.2%})")
print(f"   Final Generation: {generation}/{MAX_GENERATION}")
print(f"   Total Runtime: {total_time/60:.2f} minutes")
print(f"   Perfect Groups: {constraint_stats['perfect_groups']}/{K} ({constraint_stats['perfect_groups']/K:.2%})")
print(f"   C1 (HTQ): {constraint_stats['C1_satisfied']}/{K} ({constraint_stats['C1_satisfied']/K:.2%})")
print(f"   C2 (Majors): {constraint_stats['C2_satisfied']}/{K} ({constraint_stats['C2_satisfied']/K:.2%})")
print(f"   C3 (Gender): {constraint_stats['C3_satisfied']}/{K} ({constraint_stats['C3_satisfied']/K:.2%})")
print(f"   C4 (Size): {constraint_stats['C4_satisfied']}/{K} ({constraint_stats['C4_satisfied']/K:.2%})")
print("="*80)